# Analysing the  Network Diagram 

Import libraries and the init the dataset fron github

In [20]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

df= pd.read_csv("https://github.com/theamitabh/graphnetwork/raw/main/datasets/dataset.csv")  #load the data set


In [21]:
df=df.sort_values(by=['user_id','date_created'])
df["source"] = df["chapter"] + df["book"] 
df["target"]= df["source"].shift(-1)   
df["user_check"]= df["user_id"].shift(-1) # user_check column by shifting up a row to identify is user_id changes in next row



Identify whether to keep or drop the row.  Returns the user_id if next_row_user (in user_check) is the same is current user_id. If not NaN 

In [22]:
df['keep_drop'] = np.where((df['user_check'] == df['user_id']) , df['user_id'], np.nan)

df=df.dropna(subset=["keep_drop"])  # build edge list my removing rows having keep_drop as null values

df.head()


,user_id,number_of_ticks,date_created,chapter,book,numberofentriesofbook,numberofusersofbook,product_manning_id,user_owns_book,source,target,user_check,keep_drop
1,-2146355313,8,"""2019-12-14 10:05:50""",chapter-8,fain3,737,225,1069,1,chapter-8fain3,chapter-1ramamurthy,-2.146355e+09,-2.146355e+09
2,-2146355313,8,"""2019-12-14 10:05:50""",chapter-1,ramamurthy,715,202,1120,1,chapter-1ramamurthy,chapter-1thomas,-2.146355e+09,-2.146355e+09
29,-2145886989,10,"""2019-12-10 13:07:05""",chapter-4,mcnamara,2533,461,880,1,chapter-4mcnamara,chapter-4mcnamara,-2.145887e+09,-2.145887e+09
32,-2145886989,10,"""2019-12-10 13:41:25""",chapter-4,mcnamara,2533,461,880,1,chapter-4mcnamara,chapter-1mcnamara,-2.145887e+09,-2.145887e+09
7,-2145886989,1,"""2019-12-10 19:37:20""",chapter-1,mcnamara,2533,461,880,1,chapter-1mcnamara,chapter-1mcnamara,-2.145887e+09,-2.145887e+09


Creating a network using [NetworkX Pandas Edge List function ](https://networkx.github.io/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html#)   and subsequent rendering

In [23]:
G=nx.from_pandas_edgelist(df,"source","target", create_using=nx.DiGraph)

For downloading the csv files locally

In [ ]:
# Colab specific
from google.colab import files
df.to_csv('data_for_gephi.csv')
files.download('data_for_gephi.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Calculate Centrality**

In [27]:
out=nx.out_degree_centrality(G)
nx.set_node_attributes(G, out, 'out-degree')

bb = nx.betweenness_centrality(G)
nx.set_node_attributes(G, bb, 'betweenness')

eigen= nx.eigenvector_centrality(G)
nx.set_node_attributes(G, eigen, 'eigen')

#List of all chapter & all centrality
data_list=list(G.nodes(data=True))

data_list[25]  # Sample  list content 



('welcomeborgatti',
 {'betweenness': 0.0011775774589375227,
  'eigen': 1.4960051529769776e-13,
  'out-degree': 0.0007132667617689016})

In [29]:
data = {}  #init a dictionary
data['chapter']=[x[0] for x in data_list]
data['out-degree'] = [x[1]['out-degree'] for x in data_list]
data['eigen'] = [x[1]['eigen'] for x in data_list]
data['betweenness'] = [x[1]['betweenness'] for x in data_list]
dfcentrality = pd.DataFrame(data)
dfcentrality.head()



,chapter,out-degree,eigen,betweenness
0,chapter-8fain3,0.000713,5.159157e-46,0.000000e+00
1,chapter-1ramamurthy,0.000713,3.405044e-44,5.091126e-07
2,chapter-1thomas,0.000000,1.107155e-42,0.000000e+00
3,chapter-4mcnamara,0.002140,7.421039e-11,1.095101e-03
4,chapter-1mcnamara,0.001427,5.105664e-11,1.540575e-03


In [34]:
row1 = [n for n in G['chapter-4mcnamara']]
df2= pd.DataFrame(row1)


df3=df2.set_index(0).join(dfcentrality.set_index('chapter'))
df3=df3.sort_values(by=['eigen'],ascending=False)
df3

,out-degree,eigen,betweenness
0,,,
chapter-4mcnamara,0.002140,7.421039e-11,0.001095
chapter-1mcnamara,0.001427,5.105664e-11,0.001541
chapter-5mcnamara,0.002140,5.105664e-11,0.000006
